In [1]:
import pandas as pd
import numpy as np

import ames as ames

In [2]:
df_train = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')

In [4]:
df_train_renamed = ames.rename(df_train)
df_test_renamed = ames.rename(df_test)

In [5]:
df_train_cleaned = ames.map(df_train_renamed)
df_test_cleaned = ames.map(df_test_renamed)

In [6]:
# Check value_counts for all categorical data

# street               # DONE          DON'T USE, 2044 / 7 feature split
# land_cont            # DONE          Max: 1843 / Min: 43
# neighborhood         # DONE          Max: 310 / Min: 1, 2, 3, 6; NEED TO CHECK MAP AND GROUP SMALL NUMBER
# cond_1               # DONE          Combined RRNn/RRNe, RRAn/RRAe
# cond_2               # DONE          DO NOT USE
# bldg_type            # DONE          Max: 1700 / Min: 46
# style                # DONE          COMBINE 1.5Fin and 2.5Fin, 1.5Unf and 2.5Unf
# overall_cond         # DONE          Max: 1168 / Min: 4; COMBINE 1, 2, 3 values
# roof_style           # DONE          Max: 1619 / Min: 3; DON'T USE AT FIRST
# exter_cond           # DONE          COMBINE Gd and Ex, Po and Fa
# foundation           # DONE          Max: 926 / Min: 2, 5; CONSIDER COMBINING Wood and Stone
# bsmt_cond            # DONE          COMBINE Gd and Ex, Po and Fa
# bsmt_fin_1           # DONE          Max: 615 / Min: 102
# bsmt_fin_2           # DONE          DON'T USE; Max: 1749 / Min: 23 
# bsmt_sf              # DONE          USE AS NUMERICAL
# heat                 # DONE          COMBINE NON-GAS METHODS (Wall, Grav, OthW)
# cent_air             # DONE          Y: 1910 / N: 141
# kitchen              # DONE          COMBINE 0 and 1, 2 and 3
# kitch_qual           # DONE          USE AS IS
# fireplaces           # DONE          COMBINE 2 and 3 and 4
# garage_type          # DONE          USE AS IS
# garage_car_size      # DONE          COMBINE 3 and 4 and 5
# paved_drive          # DONE          DON'T USE AT FIRST, USE AS IS
# has_pool             # DONE          0: 2042 / 1: 9
# sale_price           # DONE          Y target

# df_train_cleaned['overall_cond'].value_counts()

5    1168
6     368
7     270
8     101
4      70
3      45
9      29
Name: overall_cond, dtype: int64

In [7]:
df_test_cleaned.to_csv('./datasets/test_cleaned.csv', index=False)
df_train_cleaned.to_csv('./datasets/train_cleaned.csv', index=False)